## Import

In [ ]:
import uproot as up
from cycler import cycler
import mplhep as hep
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path

## Define common variables

In [ ]:
colorDict = {
    'DY+jets': '#99ffff',
    r'$H\rightarrow b\bar{b}$': '#ccccff',
    'QCD': '#ffccff',
    'Diboson': '#ffff99',
    'Single t': '#ff9999',
    r'$t\bar{t}$': '#ffcc66',
    'W+jets': '#ccffcc',
    'Z+jets':'#99ffff'
}

processNames = {
    'qcdMC': 'QCD',
    'qcd': 'QCD',
    'tt': r'$t\bar{t}$',
    'ttMC': r'$t\bar{t}$',
    'stMC': 'Single t',
    'st': 'Single t',
    'vvMC': 'Diboson',
    'vv': 'Diboson',
    'hbbMC': r'$H\rightarrow b\bar{b}$',
    'hbb': r'$H\rightarrow b\bar{b}$',
    'dyjetsMC': 'DY+jets',
    'wjets': 'W+jets',
    'wjetsMC': 'W+jets',
    'zjets': 'Z+jets'
    #'Mhs_50': 'Signal'
}

mc_labels40to120 = ['DY+jets','Z+jets', r'$H\rightarrow b\bar{b}$', 
                    'QCD', 'Diboson', 'Single t', r'$t\bar{t}$', 'W+jets']
mc_labels120to300 = ['DY+jets', 'Z+jets', r'$H\rightarrow b\bar{b}$', 
                     'QCD', 'Diboson', 'Single t', r'$t\bar{t}$', 'W+jets']

# Make postfit plots per recoil bin

## Draw postfit plot per recoil bin

In [ ]:
def plot_postfit(f, region, year, category, recoil, processes40to120, processes120to300, saveplot=False):
    
    print(region, year, category, recoil)
    
    identifier40to120 = region+year+category+'mass40to120recoil'+recoil
    identifier120to300 = region+year+category+'mass120to300recoil'+recoil

    prefit_dir40to120 = f[identifier40to120+'_prefit']
    postfit_dir40to120 = f[identifier40to120+'_postfit']

    prefit_dir120to300 = f[identifier120to300+'_prefit']
    postfit_dir120to300 = f[identifier120to300+'_postfit']
    
    totbkg40to120 = prefit_dir40to120["TotalBkg"].values
    edges40to120 = prefit_dir40to120["TotalBkg"].edges

    totbkg120to300 = prefit_dir120to300["TotalBkg"].values
    edges120to300 = prefit_dir120to300["TotalBkg"].edges[1:] ## to remove duplicate edge
    
    totbkg = np.concatenate((totbkg40to120, totbkg120to300), axis=None)
    edges = np.concatenate((edges40to120, edges120to300), axis=None)

    fig, (ax, rax) = plt.subplots(2, 1, figsize=(10,10), 
                                  gridspec_kw=dict(height_ratios=[3, 1], hspace=0.07), sharex=True)
    errps = {'hatch':'////', 'facecolor':'none', 'lw': 0, 'color': 'k', 'alpha': 0.3}
    ax.set_ylabel('Events/GeV', fontsize=15)
    ax._get_lines.prop_cycler = ax._get_patches_for_fill.prop_cycler
    args = {'linestyle':'--', 'linewidth': 5}
    ax.set_yscale('log')
    ax.set_ylim(1e-2, 5e+5)
    hep.cms.label(ax=ax, loc=0, lumi=lumi, year=year, fontsize=15)

    ### Move to draw postfit
    sum_prefit = np.zeros(len(edges)-1)
    sum_postfit = np.zeros(len(edges)-1)
    sum_postfit_variance = np.zeros(len(edges)-1)
    process_bin = []
    mc_list = []

    keys40to120 = []
    for i in postfit_dir40to120.keys():
        keys40to120.append(str(i).replace('b\'','').replace(';1\'',''))
    keys120to300 = []
    for i in postfit_dir120to300.keys():
        keys120to300.append(str(i).replace('b\'','').replace(';1\'',''))

    for (i, j) in zip(processes40to120, processes120to300):
        #print('Which process', i, j)
        if not i in keys40to120:
            print('Not found, skip this process')
            print('Remove its label too \n')
            continue
        else:
            prefit40to120 = prefit_dir40to120[i].values
            postfit40to120 = postfit_dir40to120[i].values
            postfit40to120_variance = np.minimum(postfit40to120, postfit_dir40to120[i].variances)

        if not j in keys120to300:
            print('Not found, skip this process')
            print('Remove its label too \n')
            continue
        else:
            prefit120to300 = prefit_dir120to300[j].values
            postfit120to300 = postfit_dir120to300[j].values
            postfit120to300_variance = np.minimum(postfit120to300, postfit_dir120to300[j].variances)

        prefit_bin = np.concatenate((prefit40to120, prefit120to300), axis=None)
        postfit_bin = np.concatenate((postfit40to120, postfit120to300), axis=None)
        postfit_bin_variance = np.concatenate((postfit40to120_variance, postfit120to300_variance), axis=None)
        sum_prefit += prefit_bin
        sum_postfit += postfit_bin
        sum_postfit_variance += postfit_bin_variance
        process_bin.append(postfit_bin.tolist())
        mc_list.append(processNames[j])

    prefit = sum_prefit
    postfit = sum_postfit
    
    colors=[]
    for x in mc_list:
        colors.append(colorDict[x])
    colors.reverse()
    ax.set_prop_cycle(cycler(color=colors))

    ### Try to draw stack plots
    hep.histplot(process_bin, edges, ax=ax, stack=True, 
                 histtype='fill', edgecolor = 'k', linewidth=1, label=mc_list)
    
    ### Draw stat uncs.
    y1 = sum_postfit - np.sqrt(sum_postfit_variance)
    y1 = np.append(y1, 0)
    y2 = sum_postfit + np.sqrt(sum_postfit_variance)
    y2 = np.append(y2, 0)
    ax.fill_between(
        x = edges,
        y1 = y1,
        y2 = y2,
        step = 'post',
        **errps, label='Unc.'
    )

    hep.histplot(postfit, edges, ax=ax, label=["SM total (post-fit)"], color='b', linewidth=3)
    hep.histplot(prefit, edges, ax=ax, label=["SM total (pre-fit)"], color='r', 
                 linestyle='dashed', linewidth=2)

    ### Call data ###
    data40to120 = postfit_dir40to120["data_obs"].values
    data120to300 = postfit_dir120to300["data_obs"].values
    data = np.concatenate((data40to120, data120to300), axis=None)
    hep.histplot(data, edges, ax=ax, histtype='errorbar', label="Data", color='k')
    
    handles, labels = ax.get_legend_handles_labels()
    order = [9,1,0,2,3,4,5,6,7,8]
    ax.legend([handles[idx] for idx in order],[labels[idx] for idx in order],
               loc='upper right', fontsize=12, ncol=2) 

    ### Drawing in ratio axes ###
    hep.histplot(data/prefit, edges, yerr=np.sqrt(data)/prefit, ax=rax, histtype='errorbar', 
                 color='r', capsize=4, label="Prefit")
    hep.histplot(data/sum_postfit, edges, yerr=np.sqrt(data)/sum_postfit, ax=rax, histtype='errorbar', 
                 color='b', capsize=4, label="Postfit")
    
    y1 = 1.- np.sqrt(sum_postfit)/sum_postfit
    y1 = np.append(y1, 0)
    y2 = 1.+ np.sqrt(sum_postfit)/sum_postfit
    y2 = np.append(y2, 0)
    
    rax.fill_between(
        x = edges,
        y1 = y1,
        y2 = y2,
        step='post',
       **errps, label='Bkg Uncs.'
    )

    rax.axhline(1, ls='--', color='k')
    rax.set_ylim(0.5, 2.0)
    rax.set_xlabel('AK15 Mass [GeV]', fontsize=15)
    rax.set_ylabel('Obs/Exp', fontsize=15)#, loc='center')
    handles, labels = rax.get_legend_handles_labels()
    order = [1,2,0]
    rax.legend([handles[idx] for idx in order],[labels[idx] for idx in order],
               loc='upper right', fontsize=12, ncol=3) 


    os.system('mkdir -p ./plots/darkhiggs/postfit/'+year+'/')
    plot_path = os.path.abspath('./plots/darkhiggs/postfit/'+year+'/')
    plot_name = region+'_'+category+'_recoil'+recoil+'.png'
    
    if saveplot:
        fig.savefig(os.path.join(plot_path, plot_name))

## Here we make postfit plots including uncertainties

In [ ]:
saveplot = True

year = '2018'

lumis = { #Values from https://twiki.cern.ch/twiki/bin/viewauth/CMS/PdmVAnalysisSummaryTable                                                      
        '2016': "35.92",
        '2017': "41.53",
        '2018': "59.74"
    }
lumi=lumis[year]

f = up.open("./darkhiggs.postfit")



#sr 
for i in range(4):
    #fail
    processes40to120 = ['hbbMC', 'qcdMC', 'vvMC', 'stMC', 'ttMC', 'wjets', 'zjets']
    processes120to300 = ['hbbMC', 'qcdMC', 'vvMC', 'stMC', 'ttMC', 'wjets', 'zjets']
    plot_postfit(f, 'sr', year, 'fail', str(i), processes40to120, processes120to300, saveplot)

    #sr pass
    processes40to120 = ['hbbMC', 'qcdMC', 'vvMC', 'stMC', 'ttMC', 'wjets', 'zjets']
    processes120to300 = ['hbbMC', 'qcdMC', 'vvMC', 'stMC', 'tt', 'wjets', 'zjets']
    plot_postfit(f, 'sr', year, 'pass', str(i), processes40to120, processes120to300, saveplot)
    
#fail
processes40to120 = ['hbbMC', 'qcdMC', 'vvMC', 'stMC', 'ttMC', 'wjets', 'zjets']
processes120to300 = ['hbbMC', 'qcdMC', 'vvMC', 'stMC', 'ttMC', 'wjets', 'zjets']
plot_postfit(f, 'sr', year, 'fail', '4', processes40to120, processes120to300, saveplot)

#sr pass
processes40to120 = ['hbbMC', 'qcdMC', 'vvMC', 'stMC', 'ttMC', 'wjetsMC', 'zjets']
processes120to300 = ['hbbMC', 'qcdMC', 'vvMC', 'stMC', 'ttMC', 'wjetsMC', 'zjets']
plot_postfit(f, 'sr', year, 'pass', '4', processes40to120, processes120to300, saveplot)

# Single postfit plot per category

## Save yields per recoil and process in pandas

In [ ]:
def save_yields_in_pandas(f, region, year, category, processes):
    data = {}

    # Return edges
    identifier40to120 = region + year + category + 'mass40to120recoil0'
    identifier120to300 = region + year + category + 'mass120to300recoil0'

    postfit_dir40to120 = f[identifier40to120 + '_postfit']
    postfit_dir120to300 = f[identifier120to300 + '_postfit']

    edges40to120 = postfit_dir40to120["TotalBkg"].edges
    edges120to300 = postfit_dir120to300["TotalBkg"].edges[1:]  # Remove duplicate edge
    edges = np.concatenate((edges40to120, edges120to300), axis=None)

    for recoil in range(5):
        identifier40to120 = region + year + category + 'mass40to120recoil' + str(recoil)
        identifier120to300 = region + year + category + 'mass120to300recoil' + str(recoil)

        prefit_dir40to120 = f[identifier40to120 + '_prefit']
        postfit_dir40to120 = f[identifier40to120 + '_postfit']
        prefit_dir120to300 = f[identifier120to300 + '_prefit']
        postfit_dir120to300 = f[identifier120to300 + '_postfit']

        keys40to120 = [str(i).replace('b\'', '').replace(';1\'', '') for i in postfit_dir40to120.keys()]
        keys120to300 = [str(i).replace('b\'', '').replace(';1\'', '') for i in postfit_dir120to300.keys()]

        for process in processes:
            if process not in keys40to120 or process not in keys120to300:
                continue

            prefit40to120 = prefit_dir40to120[process].values
            postfit40to120 = postfit_dir40to120[process].values
            postfit40to120_variance = np.minimum(postfit40to120, postfit_dir40to120[process].variances)

            prefit120to300 = prefit_dir120to300[process].values
            postfit120to300 = postfit_dir120to300[process].values
            postfit120to300_variance = np.minimum(postfit120to300, postfit_dir120to300[process].variances)

            prefit_bin = np.concatenate((prefit40to120, prefit120to300), axis=None)
            postfit_bin = np.concatenate((postfit40to120, postfit120to300), axis=None)
            postfit_bin_variance = np.concatenate((postfit40to120_variance, postfit120to300_variance), axis=None)

            name = process.replace('MC', '')
            data[f"{name}_recoil{recoil}_prefit"] = prefit_bin
            data[f"{name}_recoil{recoil}_postfit"] = postfit_bin
            data[f"{name}_recoil{recoil}_var"] = postfit_bin_variance

        # Data
        data40to120 = postfit_dir40to120["data_obs"].values
        data120to300 = postfit_dir120to300["data_obs"].values
        data_contents = np.concatenate((data40to120, data120to300), axis=None)
        data[f"data_recoil{recoil}_obs"] = data_contents

    yields_df = pd.DataFrame.from_dict(data)
    outdir = Path(f'./{region}_{year}_{category}')
    outdir.mkdir(exist_ok=True)
    outfile = outdir / "yields.csv"
    yields_df.to_csv(outfile, index=False)
    
    edges_df = pd.DataFrame(edges, columns = ['edges'])
    outfile = outfile = outdir / "edges.csv"
    edges_df.to_csv(outfile, index=False)

In [ ]:
year = '2016'
region = 'sr'

processes = ['hbbMC', 'qcdMC', 'vvMC', 'stMC', 'tt', 'ttMC', 'wjets', 'wjetsMC', 'zjets']
f = up.open("./darkhiggs.postfit")

edges = None

for cate in ['pass', 'fail']:
    file = Path(f'./{region}_{year}_{cate}/yields.csv')
    if not file.exists():
        save_yields_in_pandas(f, region, year, cate, processes)

## Make plots by reading yields from dataframe

In [ ]:
yields_df = pd.read_csv('sr_2016_fail/yields.csv')
edges_df = pd.read_csv('sr_2016_fail/edges.csv')
edges = edges_df['edges']

processes = ['hbb', 'qcd', 'vv', 'st', 'tt', 'wjets', 'zjets']

lumis = {
    '2016': "35.92",
    '2017': "41.53",
    '2018': "59.74"
}
lumi = lumis[year]

postfit_bins = []
prefit = np.zeros(yields_df.shape[0])
postfit = np.zeros(yields_df.shape[0])
var = np.zeros(yields_df.shape[0])

mc_list = []

# Sum by process
for process in processes:
    prefit_columns = [col for col in yields_df.columns if col.startswith(process) and col.endswith('_prefit')]
    prefit_sum = yields_df[prefit_columns].sum(axis=1)
    prefit += prefit_sum

    postfit_columns = [col for col in yields_df.columns if col.startswith(process) and col.endswith('_postfit')]
    postfit_sum = yields_df[postfit_columns].sum(axis=1)
    postfit += postfit_sum
    postfit_bins.append(postfit_sum)

    var_columns = [col for col in yields_df.columns if col.startswith(process) and col.endswith('_var')]
    var_sum = yields_df[var_columns].sum(axis=1)
    var += var_sum

    mc_list.append(processNames[process])

# Sum data
data_columns = [col for col in yields_df.columns if col.startswith('data') and col.endswith('_obs')]
data_sum = yields_df[data_columns].sum(axis=1)

# Apply color scheme
colors = [colorDict[x] for x in mc_list[::-1]]

# Start drawing
fig, (ax, rax) = plt.subplots(2, 1, figsize=(10, 10), 
                              gridspec_kw=dict(height_ratios=[3, 1], hspace=0.07), sharex=True)
errps = {'hatch': '////', 'facecolor': 'none', 'lw': 0, 'color': 'k', 'alpha': 0.3}
ax.set_ylabel('Events/GeV', fontsize=15)
ax._get_lines.prop_cycler = ax._get_patches_for_fill.prop_cycler
args = {'linestyle': '--', 'linewidth': 5}
ax.set_yscale('log')
ax.set_ylim(1e-2, 2e+6)
hep.cms.label(ax=ax, loc=0, lumi=lumi, year=year, fontsize=15)
ax.set_prop_cycle(cycler(color=colors))

hep.histplot(data_sum, edges, ax=ax, histtype='errorbar', label="Data", color='k')
hep.histplot(prefit, edges, ax=ax, label=["SM total (pre-fit)"], color='r', linestyle='dashed', linewidth=2)
hep.histplot(postfit, edges, ax=ax, label=["SM total (post-fit)"], color='b', linewidth=3)
hep.histplot(postfit_bins, edges, ax=ax, stack=True, histtype='fill', edgecolor='k', linewidth=1, label=mc_list)

# Draw stat uncs.
y1 = postfit - np.sqrt(var)
y1 = np.append(y1, 0)
y2 = postfit + np.sqrt(var)
y2 = np.append(y2, 0)
ax.fill_between(x=edges, y1=y1, y2=y2, step='post', **errps, label='Unc.')

handles, labels = ax.get_legend_handles_labels()
order = [10, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
ax.legend([handles[idx] for idx in order], [labels[idx] for idx in order], 
          loc='upper right', fontsize=12, ncol=3)

# Drawing in ratio axes
hep.histplot(data_sum / prefit, edges, yerr=np.sqrt(data_sum) / prefit, ax=rax, histtype='errorbar', color='r', capsize=4, label="Prefit")
hep.histplot(data_sum / postfit, edges, yerr=np.sqrt(data_sum) / postfit, ax=rax, histtype='errorbar', color='b', capsize=4, label="Postfit")

y1 = 1. - np.sqrt(var) / var
y1 = np.append(y1, 0)
y2 = 1. + np.sqrt(var) / var
y2 = np.append(y2, 0)

rax.fill_between(x=edges, y1=y1, y2=y2, step='post', **errps, label='Bkg Uncs.')

rax.axhline(1, ls='--', color='k')
rax.set_ylim(np.amax(y1)-0.2, 2.0)
rax.set_xlabel('AK15 Mass [GeV]', fontsize=15)
rax.set_ylabel('Obs/Exp', fontsize=15)
rax.grid(axis='y')
handles, labels = rax.get_legend_handles_labels()
order = [1, 2, 0]
rax.legend([handles[idx] for idx in order], [labels[idx] for idx in order], 
           loc='upper right', fontsize=12, ncol=3)